### 1.1 Create Thread and Run

In [8]:
%%time
import threading

def action(max):
    for i in range(max):
        # use current_thread() function to get the current running threading
        # use getName() function to get the name of current running threading
        print(threading.current_thread().getName() + " " + str(i))
        
# the main() body
for i in range(10):
    print(threading.current_thread().getName() + " " + str(i))
    if i == 2:
        # Create and run the first thread
        t1 = threading.Thread(target = action, args = (10,))
        t1.start()
        # Create and run the first thread
        t2 = threading.Thread(target = action, args = (10,))
        t2.start()

print('Task Complete!')

MainThread 0
MainThread 1
MainThread 2
Thread-16 0
Thread-16 1
Thread-16 2
Thread-16 3
Thread-16 4
Thread-16 5
Thread-16 6
Thread-16 7
Thread-16 8
Thread-16 9
Thread-17 0MainThread 3
Thread-17 1
Thread-17 2
Thread-17 3
Thread-17 4
Thread-17 5
Thread-17 6
Thread-17 7
Thread-17 8
Thread-17 9

MainThread 4
MainThread 5
MainThread 6
MainThread 7
MainThread 8
MainThread 9
Task Complete!
Wall time: 3.99 ms


Although the above script only creates 2 threads, actually it works as 3. Because when we run the python script, **it would automatically create at least one Main Thread**

### 1.2 Create a Local Class inherited by Thread Class
Define the subclass of Threads, we **need to overwrite it's `run()` function**, **the body of `run()` represents the task each thread need to complete.**

In [10]:
%%time
# Crate a subclass of Thread
class YlThread(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        self.i = 0
    
    # overwrite it's run() function as the main body
    def run(self):
        while self.i< 5:
            print (threading.current_thread().getName() + " " + str(self.i))
            self.i +=1

for i in range(10):
    print(threading.current_thread().getName() + " " + str(i))
    if i == 2:
        ft1 = YlThread()
        ft1.start()
        ft2 = YlThread()
        ft2.start()
print('Task Finished')

MainThread 0
MainThread 1
MainThread 2
Thread-20 0
Thread-20 1
Thread-20 2
Thread-20 3
Thread-20 4
Thread-21 0
Thread-21 1
Thread-21 2
Thread-21 3
Thread-21 4
MainThread 3
MainThread 4
MainThread 5
MainThread 6
MainThread 7
MainThread 8
MainThread 9
Task Finished
Wall time: 3.99 ms


### 1.3 The Life Circule of Threads 
**The life circule of a thread could be divided into**
* New
* Ready
* Running
* Blocked
* Dead

**If you directly call `run()` within a Thread object, CPU would execute line by line instead of parallel.**

In [11]:
def action(max):
    for i in range(max):
        # use current_thread() function to get the current running threading
        # use getName() function to get the name of current running threading
        print(threading.current_thread().name + " " + str(i))
        
for i in range(10):
    print(threading.current_thread().name + " " + str(i))
    if i == 2:
        # These two lines would not create 2 threads
        threading.Thread(target=action, args = (5,)).run()
        threading.Thread(target=action, args = (5,)).run()

MainThread 0
MainThread 1
MainThread 2
MainThread 0
MainThread 1
MainThread 2
MainThread 3
MainThread 4
MainThread 0
MainThread 1
MainThread 2
MainThread 3
MainThread 4
MainThread 3
MainThread 4
MainThread 5
MainThread 6
MainThread 7
MainThread 8
MainThread 9


When **a thread complete task or catch some error**, it would go dead. **Do not want to use the `start()` function to call it again.** Dead means it would never work again.

In [14]:
# example for calling a thread already dead
sd = threading.Thread(target= action, args = (5,))
for i in range(20):
    print(threading.current_thread().name + " " + str(i))
    if i == 20:
        sd.start()
        print(sd.is_alive())
if i > 20 and not(sd.is_alive()):
    sd.start()

MainThread 0
MainThread 1
MainThread 2
MainThread 3
MainThread 4
MainThread 5
MainThread 6
MainThread 7
MainThread 8
MainThread 9
MainThread 10
MainThread 11
MainThread 12
MainThread 13
MainThread 14
MainThread 15
MainThread 16
MainThread 17
MainThread 18
MainThread 19


### 1.4 Manipulate Threads
* `join()` function
* `sleep()` function

In [15]:
threading.Thread(target=action,args=(10,),name="New Thread").start()
for i in range(10):
    if i == 2:
        jt = threading.Thread(target=action,args=(10,),name = "Joined Thread")
        jt.start()
        # In the MainThread we call the join() under jt thread
        # So the MainThread would wait until jt Thread complete all tasks
        jt.join()
    print(threading.current_thread().name + " " + str(i))

New Thread 0
New Thread 1
New Thread 2
New Thread 3
New Thread 4
New Thread 5
New Thread 6
New Thread 7
New Thread 8
New Thread 9
MainThread 0
MainThread 1
Joined Thread 0
Joined Thread 1
Joined Thread 2
Joined Thread 3
Joined Thread 4
Joined Thread 5
Joined Thread 6
Joined Thread 7
Joined Thread 8
Joined Thread 9
MainThread 2
MainThread 3
MainThread 4
MainThread 5
MainThread 6
MainThread 7
MainThread 8
MainThread 9


In [19]:
import time
for i in range(5):
    print("Current time : %s" %time.ctime())
    jt = threading.Thread(target=action,args=(5,))
    if i == 2:
        jt.start()
    time.sleep(1)
    print(threading.current_thread().getName() + " " + str(i))

Current time : Sat Apr 11 11:47:38 2020
MainThread 0
Current time : Sat Apr 11 11:47:39 2020
MainThread 1
Current time : Sat Apr 11 11:47:40 2020
Thread-35 0
Thread-35 1
Thread-35 2
Thread-35 3
Thread-35 4
MainThread 2
Current time : Sat Apr 11 11:47:41 2020
MainThread 3
Current time : Sat Apr 11 11:47:42 2020
MainThread 4


### 1.5 Concurrency Problem

In [26]:
class Account:
    # define a constructor
    def __init__(self,account_no,balance):
        self.account_no = account_no
        self.balance = balance

# Define a function to simulate withdraw money from a bank
def draw(account,draw_amount):
    # sufficient balance
    if account.balance >= draw_amount:
        print(threading.current_thread().name + " withdraw successful ： " + str(draw_amount))
        time.sleep(0.001)
        # modify the balance
        account.balance -= draw_amount
        print("\n Balance is : " + str(account.balance))
    else:
        print(threading.current_thread().name + " withdraw failed! insufficient balance!")        

In [27]:
# Create a account
acct = Account("yyl",1000)
# Use two thread to withdraw money simultaneously
threading.Thread(name = 'yxy',target= draw,args=(acct,800)).start()
threading.Thread(name = 'ywu',target= draw,args=(acct,800)).start()

yxy withdraw successful ： 800
ywu withdraw successful ： 800

 Balance is : 200

 Balance is : -600


You would find the balance becomes negative in the end. For preventing this situation, **Thread module provides `Lock` and `RLock` two classes**
* `acquire(blocking = True, timeout = -1)` Lock `Lock` or `RLock` objects within `timeout` seconds.
* `release()` release those objects.
---

#### Using `Lock` class to get the following features
* The objects created from this class allows multiple access.
* No matter what function is called from this object, you would get the right answer.

In [42]:
class Account:
    def __init__(self,account_no,balance):
        # Encapsulate two variables
        self.account_no = account_no
        self._balance = balance
        self.lock = threading.RLock()
        
    # Define a function to update balance
    def getBalance(self):
        return self._balance
    def draw(self,draw_amount):
        # Lock
        self.lock.acquire()
        try:
            if self._balance >= draw_amount:
                print(threading.current_thread().name + " withdraw successful ： " + str(draw_amount))
                time.sleep(0.001)
                # update balance
                self._balance -= draw_amount
                print("\n Balance is : " + str(self._balance))
            else:
                print(threading.current_thread().name + " withdraw failed! insufficient balance!")       
        finally:
            # Complete updating, release the lock
            self.lock.release()

In [43]:
def draw(account,draw_amount):
    account.draw(draw_amount)

# Create a account
acct = Account("yyl",1000)
# Use two thread to withdraw money simultaneously
threading.Thread(name = 'yxy',target= draw,args=(acct,800)).start()
threading.Thread(name = 'ywu',target= draw,args=(acct,800)).start()

yxy withdraw successful ： 800

 Balance is : 200
ywu withdraw failed! insufficient balance!


### 1.6 Thread Pool
* `ThreadPoolExecutor` for building a thread pool
* `ProcessPoolExecutor` for building a processor pool

#### Exectuor provides following functions
* `submit(fn,*args,**kwargs)` Pass fn function to the thread pool
* `map(func,*iterables, timeout = None, chunksize = 1)` Asynchronous execute multiple threads
* `shutdown(wait = True)` Shunt down the thread pool

In [46]:
%%time
from concurrent.futures import ThreadPoolExecutor

def action(max):
    my_sum = 0
    for i in range(max):
        print(threading.current_thread().name + ' ' + str(i))
        my_sum += i
    return my_sum

# Create a thread pool with size 2
pool = ThreadPoolExecutor(max_workers = 2)

# Submit one mission to the thread pool, 10 as the input parameter
future1 = pool.submit(action,10)

# Submit another mission to the thread pool, 20 as the input parameter
future2 = pool.submit(action,20)

# Check if future1 are finished
print(future1.done())
time.sleep(1)

# Check if future2 are finished
print(future2.done())

# Check their result
print(future1.result())
print(future2.result())

# Shutdown the pool
pool.shutdown()

ThreadPoolExecutor-2_0 0
ThreadPoolExecutor-2_0 1
ThreadPoolExecutor-2_0 2
ThreadPoolExecutor-2_0 3
ThreadPoolExecutor-2_0 4ThreadPoolExecutor-2_1 0False
ThreadPoolExecutor-2_1 1
ThreadPoolExecutor-2_1 2

ThreadPoolExecutor-2_1 3
ThreadPoolExecutor-2_1 4
ThreadPoolExecutor-2_1 5
ThreadPoolExecutor-2_1 6
ThreadPoolExecutor-2_1 7
ThreadPoolExecutor-2_1 8
ThreadPoolExecutor-2_1 9
ThreadPoolExecutor-2_1 10
ThreadPoolExecutor-2_1 11
ThreadPoolExecutor-2_1 12
ThreadPoolExecutor-2_1 13
ThreadPoolExecutor-2_1 14
ThreadPoolExecutor-2_1 15
ThreadPoolExecutor-2_1 16
ThreadPoolExecutor-2_1 17
ThreadPoolExecutor-2_1 18
ThreadPoolExecutor-2_1 19

ThreadPoolExecutor-2_0 5
ThreadPoolExecutor-2_0 6
ThreadPoolExecutor-2_0 7
ThreadPoolExecutor-2_0 8
ThreadPoolExecutor-2_0 9
True
45
190
Wall time: 1 s


We could **use `result()` function to get the return from one thread, but it would block other threads until finish the return.** if you do not want to block others, could **use `add_done_callback()` function under `Future` class to add a callback function.** When threads finish their task, they would execute the callback function automatically.

In [47]:
%%time
# Create a pool with size 2
with ThreadPoolExecutor(max_workers=2) as pool:
    future1 = pool.submit(action,10)
    future2 = pool.submit(action,20)
    
    def get_result(future):
        print(future.result())
    
    # Add callback function to threads
    future1.add_done_callback(get_result)
    future2.add_done_callback(get_result)
    print("-----------")

ThreadPoolExecutor-3_0 0
ThreadPoolExecutor-3_0 1
ThreadPoolExecutor-3_0 2
ThreadPoolExecutor-3_0 3
ThreadPoolExecutor-3_0 4
ThreadPoolExecutor-3_0 5
ThreadPoolExecutor-3_0 6
ThreadPoolExecutor-3_0 7
ThreadPoolExecutor-3_0 8
ThreadPoolExecutor-3_0 9
ThreadPoolExecutor-3_0 0
ThreadPoolExecutor-3_0 1
ThreadPoolExecutor-3_0 2
ThreadPoolExecutor-3_0 3
ThreadPoolExecutor-3_0 4
ThreadPoolExecutor-3_0 5
ThreadPoolExecutor-3_0 6
ThreadPoolExecutor-3_0 7
ThreadPoolExecutor-3_0 8
ThreadPoolExecutor-3_0 9
ThreadPoolExecutor-3_0 10
ThreadPoolExecutor-3_0 11
ThreadPoolExecutor-3_0 12
ThreadPoolExecutor-3_0 13
ThreadPoolExecutor-3_0 14
ThreadPoolExecutor-3_0 15
ThreadPoolExecutor-3_0 16
ThreadPoolExecutor-3_0 17
ThreadPoolExecutor-3_0 18
ThreadPoolExecutor-3_0 19
45
190
-----------
Wall time: 3.97 ms


#### `map()` function would automatically create threads for the iterable inputs.  (Using `with open` to close pool automatically)

In [48]:
%%time
with ThreadPoolExecutor(max_workers=4) as pool:
    # The input tuple has three components, so it would create 3 threads
    results = pool.map(action,(5,10,15))
    print('---------------')
    
for r in results:
    print(r)

ThreadPoolExecutor-4_0 0
ThreadPoolExecutor-4_0 1ThreadPoolExecutor-4_1 0
ThreadPoolExecutor-4_1 1
ThreadPoolExecutor-4_1 2
ThreadPoolExecutor-4_1 3
ThreadPoolExecutor-4_1 4
ThreadPoolExecutor-4_0 2
ThreadPoolExecutor-4_0 3
ThreadPoolExecutor-4_0 4

ThreadPoolExecutor-4_1 5
ThreadPoolExecutor-4_1 6
ThreadPoolExecutor-4_1 7
ThreadPoolExecutor-4_1 8
ThreadPoolExecutor-4_1 9
ThreadPoolExecutor-4_1 0
ThreadPoolExecutor-4_1 1
ThreadPoolExecutor-4_1 2
ThreadPoolExecutor-4_1 3
ThreadPoolExecutor-4_1 4
ThreadPoolExecutor-4_1 5
ThreadPoolExecutor-4_1 6
ThreadPoolExecutor-4_1 7
ThreadPoolExecutor-4_1 8
ThreadPoolExecutor-4_1 9
ThreadPoolExecutor-4_1 10
ThreadPoolExecutor-4_1 11
ThreadPoolExecutor-4_1 12
ThreadPoolExecutor-4_1 13
ThreadPoolExecutor-4_1 14
---------------
10
45
105
Wall time: 3.99 ms


### 1.7 Multiple Processes
* Use `fork()` under the `os` module could create a subprocess (**Not Work on Windows System**)

In [50]:
# import os

# print('FathreProcess(%s) has started!' % os.getpid())

# # Create a subprocess using fork, which means the following scripts would be executed in two different threads
# pid = os.fork()
# # If pig equal to 0, means it in the subprocess
# if pig == 0:
#     print('SubProcess, with ID %s, FatherProcess is %s' %(os.getpid(),os.getppid()))
# else:
#     print('SubProcess ID is %s' % pid)
# print('Finished!')

* **Use`multiprocessing.Process` to create a subprocess**

In [52]:
import multiprocessing
class MyProcess(multiprocessing.Process):
    def __init__(self,max):
        self.max = max
        super().__init__()
    # rewrite the run function
    def run(self):
        for i in range(self.max):
            print("SubProcess(%s), FatherProcess(%s); %d" % (os.getpid(),os.getppid(),i))
if __name__ == '__main__':
    for i in range(10):
        print("FatherProcess(%s) : %d" %(os.getpid(),i))
        if i == 5:
            mp1 = MyProcess(10)
            mp1.start()
            mp2 = MyProcess(10)
            mp2.start()
            mp2.join()
    print('Task finished')